In [118]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets
import plotly.express as px
import json

In [119]:
df_fortaleza = pd.read_csv("datasets\\CVLI_Fortaleza.csv")

with open("datasets\\população.json", "r", encoding="utf-8") as f:
    populacao_2022 = json.load(f)

with open("datasets\\bairros.json", "r", encoding="utf-8") as f:
    bairros_por_ais = json.load(f)

In [120]:
populacao_total_ais = {
    ais: sum(populacao_2022.get(bairro, 0) for bairro in bairros)
    for ais, bairros in bairros_por_ais.items()
}


type(populacao_total_ais["1"])

int

In [121]:
def agrupar_por_ais(df:pd.DataFrame, nome_ais:str):
    df_filtrada = df[df["AIS"] == nome_ais]
    df_agrupada = df_filtrada.groupby("Ano").size().reset_index(name="Total")
    return df_agrupada

In [122]:
def plot_serie_temporal(df_para_plot:pd.DataFrame, ais:str):
    df_para_plot = agrupar_por_ais(df_para_plot, ais)

    plt.figure(figsize=(16, 6))
    sns.barplot(data=df_para_plot,
                x="Ano",
                hue="Ano",
                y="Total",
                legend=False,
                palette="coolwarm"
                )
    plt.title(f"Quantidade de Casos por Ano da {ais}")
    plt.xlabel("Ano")
    plt.ylabel("Quantidade de Casos")
    plt.show()

In [123]:
def serie_temporal_interativa(df: pd.DataFrame, ais: str):
    # Agrupar os dados da AIS
    df = agrupar_por_ais(df, ais)
    numero_ais = ais.split()[1]
    # Obter a população da AIS
    populacao = populacao_total_ais.get(str(int(numero_ais)))

    # Formatar o título com a população
    populacao_formatada = f"{populacao:,}".replace(",", ".")
    titulo = f"Quantidade de Casos por Ano - {ais} (População: {populacao_formatada} habitantes, de acordo com o Censo do IBGE 2022)"

    
    fig = px.bar(
        df,
        x="Ano",
        y="Total",
        color="Total",
        title=titulo,
        labels={"Total": "Quantidade de Casos"},
        height=600
    )

    fig.update_layout(showlegend=False)
    fig.show()


In [124]:
def plotar_series_temporais(df: pd.DataFrame):
    # Filtra valores válidos
    df = df[df["AIS"] != "AIS não identificadas"]
    
    ais_options = sorted(df['AIS'].unique())
    dropdown = widgets.Dropdown(options=ais_options, description='AIS:')
    
    interact(lambda ais: serie_temporal_interativa(df, ais), ais=dropdown)

In [125]:
df_fortaleza["Data"] = pd.to_datetime(df_fortaleza["Data"], format="%Y-%m-%d", errors="coerce")

In [126]:
df_fortaleza["Ano"] = df_fortaleza["Data"].dt.year

In [127]:
plotar_series_temporais(df_fortaleza)

interactive(children=(Dropdown(description='AIS:', options=('AIS 01', 'AIS 02', 'AIS 03', 'AIS 04', 'AIS 05', …